# IMPORTS

In [ ]:
from setup_classes import SCIM, AADDatabricksAPI, DatabricksPermissionsAPI
#!pip install databricks-api
from databricks_api import DatabricksAPI
import json
import configparser
from helper_functions import *

# PARAMETERS

In [ ]:
CONFIG_FILE = "./config.json"
PROFILES_FILE = "/Users/Daniel.Perez/.databrickscfg"

PAT_TOKEN_LIFETIME = -1
TOKEN_PERMISSION = "CAN_USE"
PROCESS_OUTPUT_FILE = "./process_output.json"

# CONFIG

In [ ]:
with open(CONFIG_FILE,"r") as f:
    config = json.load(f)

In [ ]:
profiles = configparser.ConfigParser()
profiles.read(PROFILES_FILE)

# MAIN

In [ ]:
central_workspace = config["central_workspace"]
satellite_workspaces = config["satellite_workspaces"]

In [ ]:
central_url = profiles[central_workspace["profile"]]["host"]
central_token = profiles[central_workspace["profile"]]["token"]
central_scope = central_workspace["secret_scope"]
central_username = central_workspace["username"]

API Objects

In [ ]:
db = DatabricksAPI(host=central_url,
                   token=central_token)
scim = SCIM(central_username,
            central_token,
            central_url)
dbPermApi = DatabricksPermissionsAPI(central_username,
                                     central_token,
                                     central_url)

Create Secret Scope to Save PAT tokens

In [ ]:
createScope(db, central_scope)

Create de PAT tokens

In [ ]:
process_output = {}

In [ ]:
for satellite_workspace in satellite_workspaces:
    sp = satellite_workspace["service_principal"]
    print(f"Processing {sp['display_name']}")
    
    print(f"\tAdding {sp['display_name']} to Databricks Workspace...")
    response = scim.ServicePrincipal.add(sp["application_id"],
                                         sp["display_name"])
    sp["scim"] = response
    print(f"\t\t {response}\n")
    
    print(f"\tGiving {TOKEN_PERMISSION} permission to Tokens API...")
    response = dbPermApi.update_token_permissions(sp["application_id"],
                                                  TOKEN_PERMISSION)
    print(f"\t\t {response}\n")
    
    print(f"\tCreating token...")
    dbAADApi = AADDatabricksAPI(sp["tenant_id"],
                                sp["application_id"],
                                sp["client_secret"],
                                central_url)
    response = dbAADApi.create_token(sp["display_name"],
                                     PAT_TOKEN_LIFETIME)
    sp["token"] = response
    print(f"\t\t {response}\n")
    
    print("\tAdding token to secret scope...")
    response = db.secret.put_secret(central_scope,
                                    sp["display_name"],
                                    sp["token"]["token_value"])
    print(f"\t\t {response}\n")
    
    process_output[sp['display_name']]=sp
    
    print(f"******************************************************\n\n")

Write output

In [ ]:
print(f"Writing output to {PROCESS_OUTPUT_FILE}...")
with open(PROCESS_OUTPUT_FILE, 'w') as f:
    json.dump(process_output, f)
    print("\t...written")

In [ ]:
scim.ServicePrincipal.list()